## Assumption

According to Skiena's note, we could start our model with a linear model likes this:
$$P(W|\sim) = f(score,\ time\ left,\ field\ position)$$ where $\sim$ are the given observation.

Possible Candidate model for getting the probibility: [Scikit General Linear Model](http://scikit-learn.org/stable/modules/linear_model.html)